In [18]:
import json
import pandas as pd
from pymongo import MongoClient
import spacy
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

from pathlib import Path
from dotenv import load_dotenv
import os

In [24]:
# get path to ENV info pass words
env_path = Path('../.env') 
load_dotenv(dotenv_path=env_path)

mongo_url = os.getenv("MONGODB_URI")
db_name = os.getenv("DB_NAME")


In [27]:
# Open server and get data
client = MongoClient(mongo_url)
db = client[db_name]
collection = db['spanish_attempts']

data = list(collection.find())

In [29]:
flattened_mistakes = []

for doc in data:
    timestamp = doc.get('timestamp')
    original_sentence = doc.get('original')
    corrected_sentence = doc.get('corrected')

    for mistake in doc.get('mistakes', []):
        category_value = mistake.get('category', None)
        if isinstance(category_value, list):
            category_value = category_value[0] if category_value else None
        flattened_mistakes.append({
            'original_sentence': original_sentence,
            'corrected_sentence': corrected_sentence,
            'original_mistake': mistake.get('original'),
            'correction': mistake.get('correction'),
            'mistake_type': mistake.get('type'),
            'category': category_value,  # take first category
            'explanation': mistake.get('explanation'),
            'timestamp': timestamp
        })

mistakes_df = pd.DataFrame(flattened_mistakes)
print(f"Total mistakes: {len(mistakes_df)}")
mistakes_df.head()
    

Total mistakes: 25


,original_sentence,corrected_sentence,original_mistake,correction,mistake_type,category,explanation,timestamp
0,Me gusta haciendo proyectos que me encantan. E...,Me gusta hacer proyectos que me encantan. Ento...,haciendo,hacer,error,None,The verb 'gustar' is followed by an infinitive...,2025-09-15 13:35:44.266
1,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir,comentar,vocabulary,vocabulary,The word 'comentar' is more appropriate here a...,2025-09-15 13:43:26.877
2,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,decir acerca de alguien,comentar sobre alguien,preposition,preposition,The correct preposition to use with 'comentar'...,2025-09-15 13:43:26.877
3,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,traer,aportar,vocabulary,vocabulary,"In this context, 'aportar' meaning 'contribute...",2025-09-15 13:43:26.877
4,Ella puede hablar acerca de sus trabajos pasad...,Ella puede hablar acerca de sus trabajos pasad...,veces,en ocasiones,vocabulary,vocabulary,'En ocasiones' meaning 'at times' fits better ...,2025-09-15 13:43:26.877


In [16]:
# load spanish nlp model
nlp = spacy.load('es_core_news_sm')

# load similarity model 
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')


C:\Users\markb\anaconda3\envs\tf-env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\markb\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back